# Analyzing the NYC Subway Dataset

Project connected to the [Udacity Intro to Data Science course](https://www.udacity.com/course/viewer#!/c-ud359-nd).

by Victor Ribeiro, October/2015

---

### Section 0. References

**About the Dataset**

Turnstile and Weather Variables dataset reports on the cumulative number of entries and exists in the NYC with additional information about the weather. 

* [Original Dataset](https://www.dropbox.com/s/meyki2wl9xfa7yk/turnstile_data_master_with_weather.csv) - data set used throughout the course and used in this report.
* [Improved Dataset](https://www.dropbox.com/s/1lpoeh2w6px4diu/improved-dataset.zip?dl=0) - cleaned-up subset of original dataset with additional variables. [Variables in the dataset](https://s3.amazonaws.com/uploads.hipchat.com/23756/665149/05bgLZqSsMycnkg/turnstile-weather-variables.pdf)

**References**

* [Mann-Whitney U Test](https://storage.googleapis.com/supplemental_media/udacityu/4332539257/MannWhitneyUTest.pdf) Udacity
* [Mann-Whitney U Test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) Wikipedia
* [Shapiro-Wilk Test](https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test) Wikipedia
* [Shapiro-Wild Test](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html) Python reference
* [Diez, David; Barr, Christopher; Çetinkaya-Rundel, Mine] OpenIntro Statistics, Third Edition

---

### Section 1. Statistical Test

**1.1 Which statistical test did you use to analyze the NYC subway data? Did you use a one-tail or a two-tail P value? What is the null hypothesis? What is your p-critical value?**

>Considering the proposed project :
>* Null hypothesis : there's no difference between number of rides in the metro while raining vs. no raining. 
>* Alternative hypothesis : there's a difference between number of rides in the metro while raining  vs.
no raining.
>
>A Mann-Whitney U Test is applied. It's a two-tail test and p-critical value is 5% (or 0.05).

---
**1.2 Why is this statistical test applicable to the dataset? In particular, consider the assumptions that the test is making about the distribution of ridership in the two samples.**

>Taking a look in the data to support decision on 1.1

In [1]:
import numpy as np
import pandas
import pandasql
import matplotlib.pyplot as plt
import datetime
import csv
import scipy
import scipy.stats
import statsmodels.api as sm
import sys
from ggplot import *
import itertools

df = pandas.read_csv("turnstile_data_master_with_weather.csv")

In [2]:
plt.figure()
bins = 25
alpha = 0.75
df[df['rain']==0]['ENTRIESn_hourly'].hist(bins = bins, alpha=alpha) 
df[df['rain']==1]['ENTRIESn_hourly'].hist(bins = bins, alpha=alpha) 
    
plt.suptitle('Histogram of ENTRIESn_hourly')
plt.ylabel('Frequency')
plt.xlabel('ENTRIESn_hourly')
plt.legend(['no rain', 'rain'])
plt.grid(True)
plt.show()

![Histogram Raining and Non-Raining](https://github.com/vfribeiro/IntroDataScience/blob/master/figure_1.png?raw=true)

>As per histogram above neither raining nor no-raining data follow a normal distribution. Indeed, by applying Shapiro-Wik test (below), we confirm datasets do not follow normal distribution as p-value for shapiro test on both raining / no-raining data is really small. 

In [4]:
print scipy.stats.shapiro(df[df['rain']==0]['ENTRIESn_hourly'])
print scipy.stats.shapiro(df[df['rain']==1]['ENTRIESn_hourly'])

(0.47661787271499634, 0.0)
(0.4715914726257324, 0.0)


>Thus, a non-parametric test (a test that does not assume the data is drawn from any particular underlying probability distribution) like Mann-Whithney U Test is applicable.

---
**1.3 What results did you get from this statistical test? These should include the following numerical values: p-values, as well as the means for each of the two samples under test.**

>Applying Mann-Whitney U Test.



In [5]:
with_rain_mean = np.mean(df[df['rain']==1]['ENTRIESn_hourly'])
without_rain_mean = np.mean(df[df['rain']==0]['ENTRIESn_hourly'])
    
U,p = scipy.stats.mannwhitneyu(df[df['rain']==1]['ENTRIESn_hourly'],
                               df[df['rain']==0]['ENTRIESn_hourly'])

print ' Mean with rain :',with_rain_mean, \
      '\n Mean without rain :', without_rain_mean, \
      '\n U :', U, \
      '\n 2*p:', 2*p

 Mean with rain : 1105.44637675 
 Mean without rain : 1090.27878015 
 U : 1924409167.0 
 2*p: 0.049999825587


**1.4 What is the significance and interpretation of these results?**

>`ENTRIESn_hourly` raining mean is slightly bigger than no-raining means. 2 \* p-value is slightly below 0.05, thus **null hypothesis is rejected**.

---


### Section 2. Linear Regression

**2.1 What approach did you use to compute the coefficients theta and produce prediction for ENTRIESn_hourly in your regression model:**
- **OLS using Statsmodels or Scikit Learn,**
- **Gradient descent using Scikit Learn,**
- **Or something different?**

>OLS (using Statsmodels) has been selected to predict `ENTRIESn_hourly`. Below functions used to run linear regression and calculate R2. `predictions` is slightly different from the one I've done in class. The new version is more suitable for running a large number of combinations.

In [6]:
# computes r2 for a given dataset and its predictions
def compute_r_squared(data, predictions):
    n = ((data - predictions)**2).sum()
    d = ((data - data.mean())**2).sum()
    
    r_squared = 1 - n/d
    return r_squared

# runs linear regression
def linear_regression(features, values):
    features = sm.add_constant(features)
    model = sm.OLS(values,features)
    results = model.fit()
    intercept = results.params[0]
    params = results.params[1:]    
    
    return intercept, params

# given a dataframe and features, calculates predictions and params
def predictions(dataframe, features):
    # Values
    values = dataframe['ENTRIESn_hourly']
    # Perform linear regression
    intercept, params = linear_regression(features, values)
    predictions = intercept + np.dot(features, params)
    
    return predictions, params

# a single run of linear regression with only one combination. Using :
# Hour : as if it's late night or rush hour may change amount of rides
# rain : as if it's raining people may take more the subway
# precipi : same as above
# meanwindspi : as if wind may lead people to take the subway
# meantempi : as if it's really hot or cold, people may decide to take the subway
# UNIT : as dummy variable
features = df[['Hour','rain','precipi','meanwindspdi','meantempi']]
dummy_units = pandas.get_dummies(df['UNIT'], prefix='unit')
features = features.join(dummy_units)

prediction, param = predictions(df, features)
compute_r_squared(df['ENTRIESn_hourly'], prediction)

0.45837555079143655

>After running multiple tests and obtaining very different results, I've decided to run some brute force to test a big number of different combinations. In order to decrease total amount of possible combinations, only means were considered for wind speedy, temperature, pressure and dewpti. All tests are including `UNIT` as dummy variable as it changes significantly R2 values.

In [7]:
# List with almost all features. 
# Following guidance on exercise 5 Set 3, EXITSn_hourly is not being used.
aall_features = ['Hour','precipi',
                 'rain','fog', 
                 'meanwindspdi',
                 'meantempi', 
                 'meanpressurei',
                 'meandewpti' ]

# multiple variables to log results
i = 0
l_rsqu = [] # log r2 for test i
l_subs = [] # log for features subsets
# global max logs and counter
r_max = -1
s_max = None 
i_max = 0
para_max = None
pred_max = None

In [8]:
# This brute force loop will select all combinations of features some specific sizes. 
for L in range(3,(len(aall_features)+1)):
    # for each possible combination, runs a linear regression
    for subset in itertools.combinations(aall_features, L):
        l_rsqu.append(i)
        l_subs.append(i)
        l_subs[i] = subset
        
        # adding selected features and dummy variable
        features = df[[subset[0]]]
        for k in range(1,len(subset)):
            features = features.join(df[[subset[k]]])
        features = features.join(pandas.get_dummies(df['UNIT'], prefix='unit'))

        # Perform linear regression
        prediction, parameters = predictions(df, features)
        l_rsqu[i] = compute_r_squared(df['ENTRIESn_hourly'], prediction)
        
        # Saving max
        if r_max < l_rsqu[i]:
            r_max = l_rsqu[i]
            s_max = subset
            i_max = i
            para_max = parameters
            pred_max = prediction
            print '\nNew max:', i_max, 'R2:', r_max, s_max
        else:
            print i, l_rsqu[i], subset
        i = i+1


New max: 0 R2: 0.457642310864 ('Hour', 'precipi', 'rain')

New max: 1 R2: 0.457691815507 ('Hour', 'precipi', 'fog')

New max: 2 R2: 0.458217834372 ('Hour', 'precipi', 'meanwindspdi')
3 ('Hour', 'precipi', 'meantempi')
4 ('Hour', 'precipi', 'meanpressurei')
5 ('Hour', 'precipi', 'meandewpti')
6 ('Hour', 'rain', 'fog')
7 ('Hour', 'rain', 'meanwindspdi')
8 ('Hour', 'rain', 'meantempi')
9 ('Hour', 'rain', 'meanpressurei')
10 ('Hour', 'rain', 'meandewpti')

New max: 11 R2: 0.458372467488 ('Hour', 'fog', 'meanwindspdi')
12 ('Hour', 'fog', 'meantempi')
13 ('Hour', 'fog', 'meanpressurei')
14 ('Hour', 'fog', 'meandewpti')
15 ('Hour', 'meanwindspdi', 'meantempi')
16 ('Hour', 'meanwindspdi', 'meanpressurei')
17 ('Hour', 'meanwindspdi', 'meandewpti')
18 ('Hour', 'meantempi', 'meanpressurei')
19 ('Hour', 'meantempi', 'meandewpti')
20 ('Hour', 'meanpressurei', 'meandewpti')
21 ('precipi', 'rain', 'fog')
22 ('precipi', 'rain', 'meanwindspdi')
23 ('precipi', 'rain', 'meantempi')
24 ('precipi', 'rain'

>After running 219 combinations, R2 is max when using the combination with all 8 pre-selected features.

In [16]:
print 'R2 Min : ', min(l_rsqu), 'R2 Max : ', max(l_rsqu), \
      'Ratio Min/Max : ',min(l_rsqu)/max(l_rsqu)

 R2 Min :  0.418493719696 R2 Max :  0.458760123729 Ratio Min/Max :  0.912227759236


>Which leaded me to run another test with all features and find another max for R2. 

In [24]:
features = df[['Hour', 'precipi', 'rain', 'fog','meanwindspdi',
                'meantempi', 'meanpressurei', 'meandewpti',
                'maxtempi', 'maxpressurei', 'mindewpti',
                'mintempi', 'minpressurei', 'maxdewpti']]
dummy_units = pandas.get_dummies(df['UNIT'], prefix='unit')
features = features.join(dummy_units)

prediction, param = predictions(df, features)
r_4all = compute_r_squared(df['ENTRIESn_hourly'], prediction)
r_4all

0.4613875439198549

>Now, the interesting part. Plotting R2 for all tests :

In [19]:
plt.figure()
plt.suptitle('Evolution of R2 for 219 tested combinations')
plt.ylabel('R2 value')
plt.xlabel('Test number')
plt.grid(True)
plt.plot(range(0,len(l_rsqu)), l_rsqu)
plt.show()

![R2 for 219 tested combinations](https://github.com/vfribeiro/IntroDataScience/blob/master/figure_5.png?raw=true)

>There is an interesting pattern on R2 behavior accross the tests according chart above. It has basically two classes of value:
>
>- A higher one between 0.455 and 0.460 for some combinations 
>- A smaller one between 0.415 and 0.420 for other combinations
>
> This leaded me to check such combinations in search of what features are presented in each case. And it turns out that if `Hour` is present, R2 will jump to `0.455-0.460`. If `Hour` is not present, R2 will have values around `0.415-0.420`. Even, if all features are used but `Hour`, R2 will drop to around `0.420`.
>
>This is curious but makes sense as checking the chart in section 3.2 shows big jumps on some specific hours of the day - notably, hours related to going to / coming back from work/school and lunch.

---
**2.2 What features (input variables) did you use in your model? Did you use any dummy variables as part of your features?**

>Brute force was used to test multiple combinations. Features with the best result considering all 219 combinations were :

In [25]:
print 'UNIT as dummy variable and :', s_max

UNIT as dummy variable and : ('Hour', 'precipi', 'rain', 'fog', 'meanwindspdi', 'meantempi', 'meanpressurei', 'meandewpti')



---
**2.3 Why did you select these features in your model? We are looking for specific reasons that lead you to believe that the selected features will contribute to the predictive power of your model.**
- **Your reasons might be based on intuition. For example, response for fog might be: “I decided to use fog because I thought that when it is very foggy outside people might decide to use the subway more often.”**
- **Your reasons might also be based on data exploration and experimentation, for example: “I used feature X because as soon as I included it in my model, it drastically improved my R2 value.”**

>I did many tests with multiple combinations. An initial one was to pick the following features : 
>* Hour : as if it's late night or rush hour may change amount of rides (chart in 3.2)
>* rain : as if it's raining people may take more the subway
>* precipi : same as above
>* meanwindspi : as if wind may lead people to take the subway
>* meantempi : as if it's really hot or cold, people may decide to take the subway
>* UNIT : as dummy variable
>
>Then, I've decided to run multiple combinations as well. To decrease total combinations possible, only means were considered for `meanwindspdi, meantempi, meanpressurei, meandewpti`. Interesting to notice that results don't change much among different combinations. 
>
>The dummy variable 'UNIT' drastically improves R2 value. As well as the feature `Hour` as mentioned on 2.1.

---

**2.4 What are the parameters (also known as "coefficients" or "weights") of the non-dummy features in your linear regression model?**

In [28]:
para_max.head(len(s_max))

Hour              67.399979
precipi          -15.191443
rain             -25.896405
fog              103.996873
meanwindspdi      23.564016
meantempi         -4.401711
meanpressurei   -208.144951
meandewpti        -1.821180
dtype: float64

---
**2.5 What is your model’s R2 (coefficients of determination) value?**

In [29]:
r_max

0.4587601237285528

---

**2.6 What does this R2 value mean for the goodness of fit for your regression model? Do you think this linear model to predict ridership is appropriate for this dataset, given this R2  value?**

> R2 is the the percentage of variance that is explained. The closer R2 is to one, the better is the model. And, the closer to zero, the worse is the model. Our R2 is smaller than 0.5 (closer to zero than to one) which is mid-term, not good, but not bad. Below, a histogram of residuals (original data - predicted data) is presented. Most of the residuals are close to zero +/- 5000.

---


In [32]:
plt.figure()
plt.suptitle('Histogram of residuals')
plt.ylabel('Frequency')
plt.xlabel('Difference original vs predicted')
plt.grid(True)
(df['ENTRIESn_hourly'] - pred_max).hist(bins = 50)
plt.show()

![Histogram of residuals](https://github.com/vfribeiro/IntroDataScience/blob/master/figure_2.png?raw=true)

### Section 3. Visualization

**Please include two visualizations that show the relationships between two or more variables in the NYC subway data.
Remember to add appropriate titles and axes labels to your plots. Also, please add a short description below each figure commenting on the key insights depicted in the figure.**

**3.1 One visualization should contain two histograms: one of  ENTRIESn_hourly for rainy days and one of ENTRIESn_hourly for non-rainy days.**
- **You can combine the two histograms in a single plot or you can use two separate plots.**
- **If you decide to use to two separate plots for the two histograms, please ensure that the x-axis limits for both of the plots are identical. It is much easier to compare the two in that case.**
- **For the histograms, you should have intervals representing the volume of ridership (value of ENTRIESn_hourly) on the x-axis and the frequency of occurrence on the y-axis. For example, each interval (along the x-axis), the height of the bar for this interval will represent the number of records (rows in our data) that have ENTRIESn_hourly that falls in this interval.**
- **Remember to increase the number of bins in the histogram (by having larger number of bars). The default bin width is not sufficient to capture the variability in the two samples.**


In [33]:
plt.figure()
bins = 20
alpha = 0.50
df[df['rain']==0]['ENTRIESn_hourly'].hist(bins = bins, alpha=alpha) 
df[df['rain']==1]['ENTRIESn_hourly'].hist(bins = bins, alpha=alpha) 
    
plt.suptitle('Histogram of ENTRIESn_hourly')
plt.ylabel('Frequency')
plt.xlabel('ENTRIESn_hourly')
plt.legend(['no rain', 'rain'])
plt.grid(True)
plt.show()

![Histogram Raining and Non-Raining](https://github.com/vfribeiro/IntroDataScience/blob/master/figure_1.png?raw=true)

---

**3.2 One visualization can be more freeform. You should feel free to implement something that we discussed in class (e.g., scatter plots, line plots) or attempt to implement something more advanced if you'd like. Some suggestions are:**
- **Ridership by time-of-day**
- **Ridership by day-of-week**

---

In [34]:
df_t1 = df[['ENTRIESn_hourly', 'Hour']].groupby('Hour').sum()
df_t1.index.name = 'Hour'
df_t1.reset_index(inplace=True)

df_t2 = df[['EXITSn_hourly', 'Hour']].groupby('Hour').sum()
df_t2.index.name = 'Hour'
df_t2.reset_index(inplace=True)

In [35]:
plt.figure()
plt.suptitle('Total entries and exits per hour of the day')
plt.ylabel('Total')
plt.xlabel('Hour of the day')
plt.grid(True)
plt.plot(df_t1['Hour'], df_t1['ENTRIESn_hourly'])
plt.plot(df_t2['Hour'], df_t2['EXITSn_hourly'])
plt.legend(['entries', 'exits'])
plt.show()

![Total entries and exits per hour of the day](https://github.com/vfribeiro/IntroDataScience/blob/master/figure_3.png?raw=true)

>Quite interesting that `EXITS` are consistently smaller than `ENTRIES` : does NYC has subway stations with no turnstiles?
>
>Trying now ggplot.

In [36]:
df_t1 = df[['ENTRIESn_hourly', 'EXITSn_hourly', 'DATEn']].groupby('DATEn').sum()
df_t1.index.name = 'DATEn'
df_t1.reset_index(inplace=True)
df_t1['DATEn'] = pandas.to_datetime(df_t1['DATEn'])
df_t1.head()

df_t2 = pandas.melt(df_t1, 'DATEn')

gg = ggplot(df_t2, aes(x='DATEn', y='value', colour = 'variable')) +\
    geom_line() +\
    ylab('Number entries or exits') +\
    xlab('') +\
    ggtitle('Total daily entries and exits') +\
    theme(axis_text_x = element_text(size=8,angle=45),
          axis_text_y = element_text(size=8))
print gg

<ggplot: (298715789)>


![Total daily entries and exits](https://github.com/vfribeiro/IntroDataScience/blob/master/figure_4.png?raw=true)

>Once more, it's possible to confirm a smaller amount of exits than entries.

### Section 4. Conclusion

**Please address the following questions in detail. Your answers should be 1-2 paragraphs long.**

**4.1 From your analysis and interpretation of the data, do more people ride
the NYC subway when it is raining or when it is not raining?**

>Yes, according the study here presented and based in the data used, it's possible to conclude with a high level of condifence that more people ride the NYC subway when it's raining than when it's not raining.

**4.2 What analyses lead you to this conclusion? You should use results from both your statistical
tests and your linear regression to support your analysis.**

>In section 1, a full statistical analysis was presented on top of the given dataset. First, data was analyzed to verify what kind of statistical test could be used. Then, after observind data does not follow a normal distribution a non-parametrical test, Mann-Whitney U Test, was selected and applied. The resulting p-value leaded to reject the null hypothesis (stating no difference between rides when it's raining vs it's not raining) with high level of confidence `(2*p-value<0.05)`.
>
>Linear regression model using OLS (Statsmodels) on top of the given dataset has generated Rˆ2 lower than 0.5 - a not good result (but also not really bad). Residual histogram shows majority of residuals (original data - predicted) are around `0 +/- 5000`.



---

### Section 5. Reflection

**Please address the following questions in detail. Your answers should be 1-2 paragraphs long.**

**5.1 Please discuss potential shortcomings of the methods of your analysis, including:**
- **Dataset,**
- **Analysis, such as the linear regression model or statistical test.**

>With respect to the data :
>
>1. **Data is from a single month : May/2011. This is a big issue for the analysis here made.** May is popular known as a 'nice wheater month' all around the globe, thus trying to figure out if people would ride more or less NYC subway using a single month does not seem reasonable. It would be better to have good data from all months during all seasons.
>2. Further data analysis, verification and fixes may be required. For instance, by reading discussions at Udacity site, it's possible to find some potential flaws (like lot amount of entries in one hour followed by immediate almost nobody some minutes after. I haven't investigated in detail the data neither discussed or verified how the data was collected.
>
>With respect to the linear regression model :
>
>* The value for R2 obtained is 0.46 - not good but not bad... Trying other models (polynomial, logistic regreassion) may lead to better results.  

---

**5.2 (Optional) Do you have any other insight about the dataset that you would like to share with us?**

>Most important comments about the dataset have been added in question above. While a great exercise (and I really enjoyed doing this project), the results in this study are useless as the data is from a single month. 

---